In [9]:
import sys
import os
import json
from langchain_core.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI
import getpass
from langchain.prompts import PromptTemplate

In [10]:
linkml_schema= """id: chinook_schema
name: Chinook
description: "LinkML schema for the Chinook database, extracted from ChinookData.json"
default_prefix: chinook
prefixes:
  chinook: "http://example.org/chinook/"
  linkml: "https://w3id.org/linkml/"

types:
  string:
    base: str
  integer:
    base: int
  number:
    base: float

enums:
  # No enums identified from the data provided

classes:
  Genre:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - genre_id
      - name
  MediaType:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - media_type_id
      - name
  Artist:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - artist_id
      - name
  Album:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - album_id
      - title
      - artist_id
  Track:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - track_id
      - name
      - composer
      - milliseconds
      - bytes
      - unit_price
      - album_id
      - media_type_id
      - genre_id
  Employee:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - employee_id
      - last_name
      - first_name
      - title
      - birth_date
      - hire_date
      - address
      - city
      - state
      - country
      - postal_code
      - phone
      - fax
      - email
      - reports_to
  Customer:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - customer_id
      - company
      - last_name
      - first_name
      - email
      - phone
      - address
      - city
      - state
      - country
      - postal_code
      - fax
      - support_rep_id
  Invoice:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - invoice_id
      - customer_id
      - invoice_date
      - billing_address
      - billing_city
      - billing_state
      - billing_country
      - billing_postal_code
      - total
  InvoiceLine:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - invoice_line_id
      - invoice_id
      - track_id
      - unit_price
      - quantity
  Playlist:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - playlist_id
      - name
  PlaylistTrack:
    description: "Extrahiert aus: ChinookData.json"
    slots:
      - playlist_id
      - track_id

slots:
  genre_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  name:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  media_type_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  artist_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  album_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  title:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  track_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  composer:
    description: "Extrahiert aus: ChinookData.json"
    range: string
    required: false
  milliseconds:
    description: "Extrahiert aus: ChinookData.json"
    range: integer
  bytes:
    description: "Extrahiert aus: ChinookData.json"
    range: integer
  unit_price:
    description: "Extrahiert aus: ChinookData.json"
    range: number
  employee_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  last_name:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  first_name:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  title:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  birth_date:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  hire_date:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  address:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  city:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  state:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  country:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  postal_code:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  phone:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  fax:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  email:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  reports_to:
    description: "Extrahiert aus: ChinookData.json"
    range: integer
    required: false
  customer_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  company:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  support_rep_id:
    description: "Extrahiert aus: ChinookData.json"
    range: integer
  invoice_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  invoice_date:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  billing_address:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  billing_city:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  billing_state:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  billing_country:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  billing_postal_code:
    description: "Extrahiert aus: ChinookData.json"
    range: string
  total:
    description: "Extrahiert aus: ChinookData.json"
    range: number
  invoice_line_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer
  track_id:
    description: "Extrahiert aus: ChinookData.json"
    range: integer
  quantity:
    description: "Extrahiert aus: ChinookData.json"
    range: integer
  playlist_id:
    description: "Extrahiert aus: ChinookData.json"
    identifier: true
    range: integer"""

system_prompt = PromptTemplate.from_template("""
Du bist ein erfahrener Data Engineer, der spezialisiert ist auf ETL-Prozesse und DataFrame-Erstellung.

Deine Aufgabe ist es, Python-Code zu generieren, der:
1. Datendateien korrekt extrahiert
2. Die Daten gemäß einem gegebenen Schema strukturiert
3. Valide pandas DataFrames erstellt
4. Den Code testet und bei Fehlern anpasst
5. Alle Ausgabedateien im "temp" Ordner speichert
5. Egal was du machst gebe dir nie die ganzen Datafrmaes aus sondern maximal immer nur 2 zeilen zum gucken fasll du das brauchst

WICHTIG: 
- Speichere ALLE erstellten DataFrames als CSV-Dateien im "temp" Ordner
- Verwende df.to_csv('temp/dataframe_name.csv', index=False) für jeden DataFrame
- Erstelle den temp Ordner falls er nicht existiert: import os; os.makedirs('temp', exist_ok=True)

Du hast Zugang zu folgenden Tools:

{tools}

Verwende das folgende Format:

Question: die ursprüngliche Frage
Thought: du solltest immer über das Problem nachdenken
Action: die Aktion aus [{tool_names}]
Action Input: der Input für die Aktion
Observation: das Ergebnis der Aktion
... (dieser Thought/Action/Action Input/Observation kann wiederholt werden)
Thought: Ich kenne jetzt die finale Antwort
Final Answer: die finale Antwort auf die ursprüngliche Frage

Beginne!

Question: {input}
Thought:{agent_scratchpad}
""")

In [11]:


parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)
from modules.schema_analyzer import SchemaAnalyzer

analyzer = SchemaAnalyzer("../knowledge_base/ChinookData.json")
result = analyzer.analyze()

number_of_lines = 10
snippets = analyzer.get_file_snippets(number_of_lines)

snippets = analyzer.get_file_snippets(n=10) # n=10 für 10 Zeilen
head_str = snippets.get('head')
middle_str = snippets.get('middle')
tail_str = snippets.get('tail')

schema_str = json.dumps(result, indent=2)

In [14]:


class QuietPythonREPLTool(PythonREPLTool):
    def _run(self, query: str) -> str:
        result = super()._run(query)
        # Schneide lange Ausgaben ab
        if len(result) > 1000:
            return result[:1000] + "\n... (Ausgabe gekürzt)"
        return result

# Verwende das neue Tool
python_tool = QuietPythonREPLTool(description="Führt Python-Code aus.")
tools = [python_tool]

# Rest bleibt gleich
llm = ChatOpenAI(temperature=0, model="gpt-4o")
agent = create_react_agent(llm=llm, tools=tools, prompt=system_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)



if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")




file_path = "../knowledge_base/ChinookData.json"
# --- 5. Interaktion mit dem Agenten ---
user_input = f"Ich will das du mir einen Code generiert welcher eine Daten File extrahiert passende pandas DataFrame. Wichtig nur ausfuehren schaue dir die Daten nicht vorher an. Hierzu geben ich dir eine LinkML file welche dazu dient die Struktur der Daten zu verstehen und die Daten in diesen Format zu uebernehemn in ein relationales Datenbankschema welches spawter in eine sql Datenbank ingested wird. Die Daten koenen Im unterschiedlichen Formaten vorliegen, wie z.B. XML, JSON, CSV oder Excel. Gib mir nur den Code also auch nicht in einem Codeblock. Die LinkML Datei ist: {linkml_schema}. The schema of the data is: {schema_str}. The file is a JSON. \n Tail of the file: {tail_str} \n Head: {head_str} \n Middle: {middle_str}"

response = agent_executor.invoke({
    "input": (
        f"Erstelle ein Python-Skript, das die Datei '{file_path}' extrahiert. "
        "Verwende das gegebene LinkML-Schema ({linkml_schema}), das die Struktur der Daten vorgibt. "
        "Die Datei ist im JSON-Format. Beispiele aus der Datei: "
        f"Head: {head_str}, Middle: {middle_str}, Tail: {tail_str}. "
        "Das Ziel ist ein korrekter DataFrame nach dem Schema: {schema_str}. "
        "Der Agent soll prüfen, ob der DataFrame gültig ist. Wenn nicht, soll er den Code anpassen und erneut versuchen."
        "Gib mir am Ende eine Zusammenfassung welcge Tballen du extrahiert hast und konntest es sollen alle die in der linkmlfile Beschreiben sind vorkommen."
    )
})

print(response)



> Entering new AgentExecutor chain...
Um die Aufgabe zu lösen, muss ich die JSON-Datei extrahieren, die Daten gemäß dem gegebenen Schema strukturieren, valide pandas DataFrames erstellen und diese im "temp" Ordner speichern. Ich werde den Prozess in mehreren Schritten durchführen.

1. Extrahiere die JSON-Datei.
2. Strukturiere die Daten gemäß dem gegebenen Schema.
3. Erstelle pandas DataFrames.
4. Speichere die DataFrames als CSV-Dateien im "temp" Ordner.
5. Teste den Code und passe ihn bei Bedarf an.

Zuerst werde ich die JSON-Datei extrahieren und die Daten in ein pandas DataFrame umwandeln.

Action: Python_REPL
Action Input:
```python
import os
import pandas as pd
import json

# Erstelle den "temp" Ordner, falls er nicht existiert
os.makedirs('temp', exist_ok=True)

# Lade die JSON-Datei
with open('../knowledge_base/ChinookData.json', 'r') as file:
    data = json.load(file)

# Zeige die ersten zwei Einträge der Daten an, um die Struktur zu verstehen
data[:2]
```
UnicodeDecodeErro